In [1]:
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests

In [2]:
### 사용자가 원하는 url 생성 및 데이터 파싱 함수
def url_parser(tmonth, uRow=5):
    uKey = "7672746155736a6436336451676175"
    
    url="http://openapi.seoul.go.kr:8088/" + uKey        # 기본 url + key
    url=url+"/xml/CardSubwayPayFree/1/"+ str(uRow) +"/"  # 데이터형식/대상/시작위치/종료위치/
    url=url+str(tmonth)                                  # 조회 년월

    url_xml = requests.get(url)
    if url_xml.status_code != 200:              # .status_code  => 200 이면 정상/ 아니면 오류
        print('데이터를 가져오지 못했습니다.')
        exit()
    soup = bs(url_xml.content, 'html.parser')
    
    return soup

In [3]:
### 딕셔너리 구조로 저장 후 DataFarame 구조로 변경
def seoul_sw_pandas(seoul_sw_soup):
    xml_row = seoul_sw_soup.find_all('row')
    xml_txt = []
    for row in xml_row:
        dm = row.find('use_mon').text               # 사용일자
        line = row.find('line_num').text            # 호선(라인)
        sub_sta = row.find('sub_sta_nm').text       # 역이름
        pay_ride = row.find('pay_ride_num').text  # 유임승차총인원
        free_ride = row.find('free_ride_num').text  # 무임승차총인원
        pay_alight = row.find('pay_alight_num').text  # 유임하차총인원
        free_alight = row.find('free_alight_num').text  # 무임하차총인원

        xml_txt.append({'사용일': dm, '라인': line, '역명': sub_sta, '유임승차': pay_ride, 
                        '무임승차':free_ride, '유임하차':pay_alight ,'무임하차':free_alight})

    #print(xml_txt)
    df = pd.DataFrame(xml_txt)
    return df

In [4]:
# 시작일부터 종료일까지의 날짜문자 리스트를 이용해 전체 일자별 데이터 조회 pandas.concat()을 이용한 행추가
def main_api(sDt, eDt):
    # pd.date_range(start=staDate, end=endDate) : 시작일부터 종료일까지 날짜 생성, 시작일/종료일=>문자형
    sDt=sDt+'01'
    eDt=eDt+"01"
    dt_index = pd.date_range(start=sDt, end=eDt,  freq='MS')
    dtList = dt_index.strftime("%Y%m").tolist()  # 날짜형을 문자형으로 변경후 리스트형으로 저장
    df0 = pd.DataFrame()  # 전체 데이터 저장 변수

    for dtl in tqdm(dtList, desc="진행율: "):
        seoul_sw_soup=url_parser(dtl)  # 한페이지에 5개의 데이터가 출력된 url 정보 가져오기
        if seoul_sw_soup.find('code').text == "INFO-200":
            continue
        uRow = seoul_sw_soup.find('list_total_count').text  # 조회된 전체 데이터 개수 추출하기
        seoul_sw_soup=url_parser(dtl, uRow)  # 한페이지에 추출한 전체 데이터 출력 url 정보 가져오기
        df = seoul_sw_pandas(seoul_sw_soup)  # 요청 데이터에 대한 DataFrame 형식으로 변경하기

        df0 = pd.concat([df0, df], ignore_index=True)  # ignore_index=True: 인덱스 재배열(재설정)

    return df0

In [5]:
if __name__ == "__main__":
    sDt = input('시작년월 입력(예: 201501): ')
    eDt = input('종료년월 입력(예: 202001): ')

    df0 = main_api(sDt, eDt)
    print('===== 작업종료 =====')

    print(df0)

시작년월 입력(예: 201501): 202101
종료년월 입력(예: 202001): 202107
['202101', '202102', '202103', '202104', '202105', '202106', '202107']


진행율: 100%|████████████████████████████████████████████████████████████████████████████| 7/7 [00:02<00:00,  2.68it/s]

===== 작업종료 =====
         사용일   라인            역명    유임승차    무임승차    유임하차    무임하차
0     202101  1호선           신설동  219283   82995  218343   79139
1     202101  1호선            시청  371942   46231  366139   43433
2     202101  1호선  청량리(서울시립대입구)  266539  205089  264050  206452
3     202101  1호선            종각  598363   84544  567563   78565
4     202101  1호선           동묘앞   98645  113919  105308  115644
...      ...  ...           ...     ...     ...     ...     ...
3634  202106  중앙선            양평   54335   28307   55227   29253
3635  202106  중앙선            덕소  115762   41318  114525   41815
3636  202106  중앙선            신원    3709    3399    2916    2837
3637  202106  중앙선            도심   50986   16499   38900   15711
3638  202106  중앙선            중랑  110574   43835  104494   42601

[3639 rows x 7 columns]
